In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [2]:
LABEL_PATH = Path("../data/labels/labels_task2.csv")
FRAME_DIR = Path("../data/frames")

df = pd.read_csv(LABEL_PATH)
available_videos = {p.name for p in FRAME_DIR.iterdir() if p.is_dir() and any(p.glob("*.jpg"))}
df = df[df["VIDEO"].isin(available_videos)].reset_index(drop=True)
print(f"Number of videos with frames: {len(df)}")

Number of videos with frames: 30


In [3]:
class OSATSDataset(Dataset):
    def __init__(self, dataframe, frame_dir, transform=None, sequence_length=16):
        self.data = dataframe.copy()
        self.frame_dir = frame_dir
        self.transform = transform
        self.sequence_length = sequence_length
        self.osats_cols = [col for col in dataframe.columns if col.startswith("OSATS_")]

        for col in self.osats_cols:
            self.data[col] = self.data[col].clip(0, 4).astype(np.int64)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        video_id = row["VIDEO"]
        y = row[self.osats_cols].values.astype(np.int64)
        path = self.frame_dir / video_id

        frames = sorted(path.glob("*.jpg"))
        selected = frames[:self.sequence_length]
        if len(selected) == 0:
            raise IndexError(f"No frames for video {video_id}")
        while len(selected) < self.sequence_length:
            selected.append(selected[-1])

        images = [self.transform(Image.open(f).convert("RGB")) for f in selected]
        return torch.stack(images), torch.tensor(y)


In [4]:
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Transforms para imagens (compatível com CNNs e ResNet)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Colunas OSATS (as 8 que vais prever)
osats_cols = [col for col in df.columns if col.startswith("OSATS_")]

# Criar uma coluna auxiliar com a média arredondada das OSATS para estratificação
df["OSATS_MEAN_LABEL"] = df[osats_cols].mean(axis=1).round().astype(int)

# Divisão 70% treino, 30% temp (com estratificação)
train_df, temp_df = train_test_split(
    df, test_size=0.30, random_state=42, stratify=df["OSATS_MEAN_LABEL"]
)

# Divisão 15% validação, 15% teste (sem stratify para evitar erro)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Criação dos datasets
train_dataset = OSATSDataset(train_df, FRAME_DIR, transform)
val_dataset = OSATSDataset(val_df, FRAME_DIR, transform)
test_dataset = OSATSDataset(test_df, FRAME_DIR, transform)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


In [5]:
from torch.nn.init import kaiming_uniform_, xavier_uniform_

class CNNModel_1(nn.Module):
    def __init__(self, num_classes=40, sequence_length=16, input_shape=(3,224,224)):
        super(CNNModel_1, self).__init__()
        self.sequence_length = sequence_length

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        conv_out_size = self._get_conv_output((sequence_length, *input_shape))
        self.fc1 = nn.Linear(conv_out_size, 100)
        kaiming_uniform_(self.fc1.weight, nonlinearity='relu')
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(100, num_classes)
        xavier_uniform_(self.fc2.weight)
        self.act2 = nn.Softmax(dim=1)

    def _get_conv_output(self, shape):
        bs = 1
        input = torch.rand(bs, *shape)
        B, T, C, H, W = input.shape
        input = input.view(B * T, C, H, W)
        output_feat = self.layer1(input)
        output_feat = self.layer2(output_feat)
        output_feat = output_feat.view(output_feat.size(0), -1)
        output_feat = output_feat.view(bs, T, -1).mean(dim=1)
        return int(np.prod(output_feat.size()[1:]))

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = out.view(B, T, -1).mean(dim=1)
        out = self.fc1(out)
        out = self.act1(out)
        out = self.fc2(out)
        out = self.act2(out)
        return out


In [6]:
class CNNModel_2(nn.Module):
    def __init__(self, num_classes=40, sequence_length=16, input_shape=(3,224,224)):
        super(CNNModel_2, self).__init__()
        self.sequence_length = sequence_length

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, 3),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        conv_out_size = self._get_conv_output((sequence_length, *input_shape))
        self.fc1 = nn.Linear(conv_out_size, num_classes)

    def _get_conv_output(self, shape):
        bs = 1
        input = torch.rand(bs, *shape)
        B, T, C, H, W = input.shape
        input = input.view(B * T, C, H, W)
        output_feat = self.layer1(input)
        output_feat = self.layer2(output_feat)
        output_feat = output_feat.view(output_feat.size(0), -1)
        output_feat = output_feat.view(bs, T, -1).mean(dim=1)
        return int(np.prod(output_feat.size()[1:]))

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = out.view(B, T, -1).mean(dim=1)
        out = self.fc1(out)
        return out


In [7]:
from torch.nn import BatchNorm2d, Dropout
import torch.nn.functional as F

class CNNModel_3(nn.Module):
    def __init__(self, num_classes=40, sequence_length=16, input_shape=(3,224,224)):
        super(CNNModel_3, self).__init__()
        self.sequence_length = sequence_length

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3),
            BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        conv_out_size = self._get_conv_output((sequence_length, *input_shape))
        self.fc1 = nn.Linear(conv_out_size, 600)
        self.drop = Dropout(0.25)
        self.fc2 = nn.Linear(600, 120)
        self.fc3 = nn.Linear(120, num_classes)

    def _get_conv_output(self, shape):
        bs = 1
        input = torch.rand(bs, *shape)
        B, T, C, H, W = input.shape
        input = input.view(B * T, C, H, W)
        output_feat = self.layer1(input)
        output_feat = self.layer2(output_feat)
        output_feat = output_feat.view(output_feat.size(0), -1)
        output_feat = output_feat.view(bs, T, -1).mean(dim=1)
        return int(np.prod(output_feat.size()[1:]))

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = out.view(B, T, -1).mean(dim=1)
        out = F.relu(self.fc1(out))
        out = self.drop(out)
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out


In [8]:
from torch.nn import Dropout2d
import torch.nn.functional as F

class CNNModel_4(nn.Module):
    def __init__(self, num_classes=40, sequence_length=16, input_shape=(3,224,224)):
        super(CNNModel_4, self).__init__()
        self.sequence_length = sequence_length

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, 5),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            Dropout2d(0.2)
        )

        conv_out_size = self._get_conv_output((sequence_length, *input_shape))
        self.fc1 = nn.Linear(conv_out_size, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def _get_conv_output(self, shape):
        bs = 1
        input = torch.rand(bs, *shape)
        B, T, C, H, W = input.shape
        input = input.view(B * T, C, H, W)
        output_feat = self.layer1(input)
        output_feat = output_feat.view(output_feat.size(0), -1)
        output_feat = output_feat.view(bs, T, -1).mean(dim=1)
        return int(np.prod(output_feat.size()[1:]))

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)
        out = self.layer1(x)
        out = out.view(out.size(0), -1)
        out = out.view(B, T, -1).mean(dim=1)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        return out


In [9]:
def train_model(model, dataloader, epochs):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            if targets.dim() > 1:
                targets = targets[:, 0]

            if outputs.shape[0] != targets.shape[0]:
                raise ValueError(f"Shape mismatch: outputs {outputs.shape} vs targets {targets.shape}")

            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(dataloader):.4f}")

    return model


In [10]:
def evaluate_model(model, dataloader):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            if targets.dim() > 1:
                targets = targets[:, 0]
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    acc = 100 * correct / total
    print(f"Accuracy: {acc:.2f}%")
    return acc

In [11]:
model1 = CNNModel_1(num_classes=40, sequence_length=16, input_shape=(3,224,224))
model1 = train_model(model1, train_loader, epochs=100)

print("Validação - Modelo 1")
evaluate_model(model1, val_loader)

print("Teste - Modelo 1")
evaluate_model(model1, test_loader)

# Guardar modelo com nome especificado
torch.save(model1.state_dict(), "../outputs/models/model_task2_cnn1.pt")


Epoch [1/100], Loss: 3.5515
Epoch [2/100], Loss: 3.3976
Epoch [3/100], Loss: 3.2726
Epoch [4/100], Loss: 3.2726
Epoch [5/100], Loss: 3.2726
Epoch [6/100], Loss: 3.2726
Epoch [7/100], Loss: 3.3976
Epoch [8/100], Loss: 3.3976
Epoch [9/100], Loss: 3.3976
Epoch [10/100], Loss: 3.3976
Epoch [11/100], Loss: 3.2726
Epoch [12/100], Loss: 3.3976
Epoch [13/100], Loss: 3.3976
Epoch [14/100], Loss: 3.3976
Epoch [15/100], Loss: 3.2726
Epoch [16/100], Loss: 3.3976
Epoch [17/100], Loss: 3.2726
Epoch [18/100], Loss: 3.3976
Epoch [19/100], Loss: 3.3976
Epoch [20/100], Loss: 3.3976
Epoch [21/100], Loss: 3.3976
Epoch [22/100], Loss: 3.2726
Epoch [23/100], Loss: 3.2726
Epoch [24/100], Loss: 3.2726
Epoch [25/100], Loss: 3.3976
Epoch [26/100], Loss: 3.2726
Epoch [27/100], Loss: 3.3976
Epoch [28/100], Loss: 3.3976
Epoch [29/100], Loss: 3.2726
Epoch [30/100], Loss: 3.2726
Epoch [31/100], Loss: 3.3976
Epoch [32/100], Loss: 3.2726
Epoch [33/100], Loss: 3.3976
Epoch [34/100], Loss: 3.3976
Epoch [35/100], Loss: 3

In [12]:
model2 = CNNModel_2(num_classes=40, sequence_length=16, input_shape=(3,224,224))
model2 = train_model(model2, train_loader, epochs=100)

print("Validação - Modelo 2")
evaluate_model(model2, val_loader)

print("Teste - Modelo 2")
evaluate_model(model2, test_loader)

torch.save(model2.state_dict(), "../outputs/models/model_task2_cnn2.pt")


Epoch [1/100], Loss: 2.2780
Epoch [2/100], Loss: 1.1471
Epoch [3/100], Loss: 1.0207
Epoch [4/100], Loss: 0.8331
Epoch [5/100], Loss: 0.6566
Epoch [6/100], Loss: 0.5938
Epoch [7/100], Loss: 0.4651
Epoch [8/100], Loss: 0.5090
Epoch [9/100], Loss: 0.3157
Epoch [10/100], Loss: 0.3054
Epoch [11/100], Loss: 0.2842
Epoch [12/100], Loss: 0.1382
Epoch [13/100], Loss: 0.1478
Epoch [14/100], Loss: 0.1138
Epoch [15/100], Loss: 0.0969
Epoch [16/100], Loss: 0.0678
Epoch [17/100], Loss: 0.0536
Epoch [18/100], Loss: 0.0413
Epoch [19/100], Loss: 0.0315
Epoch [20/100], Loss: 0.0286
Epoch [21/100], Loss: 0.0262
Epoch [22/100], Loss: 0.0334
Epoch [23/100], Loss: 0.0216
Epoch [24/100], Loss: 0.0203
Epoch [25/100], Loss: 0.0154
Epoch [26/100], Loss: 0.0149
Epoch [27/100], Loss: 0.0125
Epoch [28/100], Loss: 0.0122
Epoch [29/100], Loss: 0.0099
Epoch [30/100], Loss: 0.0092
Epoch [31/100], Loss: 0.0084
Epoch [32/100], Loss: 0.0101
Epoch [33/100], Loss: 0.0070
Epoch [34/100], Loss: 0.0070
Epoch [35/100], Loss: 0

In [13]:
model3 = CNNModel_3(num_classes=40, sequence_length=16, input_shape=(3,224,224))
model3 = train_model(model3, train_loader, epochs=100)

print("Validação - Modelo 3")
evaluate_model(model3, val_loader)

print("Teste - Modelo 3")
evaluate_model(model3, test_loader)

torch.save(model3.state_dict(), "../outputs/models/model_task2_cnn3.pt")


Epoch [1/100], Loss: 4.2357
Epoch [2/100], Loss: 1.9221
Epoch [3/100], Loss: 1.1928
Epoch [4/100], Loss: 1.5472
Epoch [5/100], Loss: 0.6307
Epoch [6/100], Loss: 0.3455
Epoch [7/100], Loss: 0.5239
Epoch [8/100], Loss: 0.4486
Epoch [9/100], Loss: 0.0568
Epoch [10/100], Loss: 0.1511
Epoch [11/100], Loss: 0.0928
Epoch [12/100], Loss: 0.2287
Epoch [13/100], Loss: 0.0276
Epoch [14/100], Loss: 0.4409
Epoch [15/100], Loss: 0.0475
Epoch [16/100], Loss: 0.0026
Epoch [17/100], Loss: 0.0180
Epoch [18/100], Loss: 0.0077
Epoch [19/100], Loss: 0.0097
Epoch [20/100], Loss: 0.0014
Epoch [21/100], Loss: 0.0012
Epoch [22/100], Loss: 0.0001
Epoch [23/100], Loss: 0.0003
Epoch [24/100], Loss: 0.0009
Epoch [25/100], Loss: 0.0009
Epoch [26/100], Loss: 0.0002
Epoch [27/100], Loss: 0.0010
Epoch [28/100], Loss: 0.0002
Epoch [29/100], Loss: 0.0018
Epoch [30/100], Loss: 0.0002
Epoch [31/100], Loss: 0.0033
Epoch [32/100], Loss: 0.0013
Epoch [33/100], Loss: 0.0006
Epoch [34/100], Loss: 0.0001
Epoch [35/100], Loss: 0

In [ ]:
model4 = CNNModel_4(num_classes=40, sequence_length=16, input_shape=(3,224,224))
model4 = train_model(model4, train_loader, epochs=100)

print("Validação - Modelo 4")
evaluate_model(model4, val_loader)

print("Teste - Modelo 4")
evaluate_model(model4, test_loader)

torch.save(model4.state_dict(), "../outputs/models/model_task2_cnn4.pt")


Epoch [1/100], Loss: 8.6049
Epoch [2/100], Loss: 8.4234
Epoch [3/100], Loss: 6.4934
Epoch [4/100], Loss: 3.9202
Epoch [5/100], Loss: 3.8293
Epoch [6/100], Loss: 0.8885
Epoch [7/100], Loss: 0.1558
Epoch [8/100], Loss: 0.0006
Epoch [9/100], Loss: 0.5408
Epoch [10/100], Loss: 0.0000
Epoch [11/100], Loss: 0.0001
Epoch [12/100], Loss: 0.0022
Epoch [13/100], Loss: 0.0641
Epoch [14/100], Loss: 0.0002
Epoch [15/100], Loss: 0.0006
Epoch [16/100], Loss: 0.0024
Epoch [17/100], Loss: 0.0018
Epoch [18/100], Loss: 0.0013
Epoch [19/100], Loss: 0.0001
Epoch [20/100], Loss: 0.0000
Epoch [21/100], Loss: 0.0000
Epoch [22/100], Loss: 0.0000
Epoch [23/100], Loss: 0.0001
Epoch [24/100], Loss: 0.0000
Epoch [25/100], Loss: 0.0000
Epoch [26/100], Loss: 0.0000
Epoch [27/100], Loss: 0.0000
Epoch [28/100], Loss: 0.0000
Epoch [29/100], Loss: 0.0000
Epoch [30/100], Loss: 0.0000
Epoch [31/100], Loss: 0.0000
Epoch [32/100], Loss: 0.0000
Epoch [33/100], Loss: 0.0000
Epoch [34/100], Loss: 0.0000
Epoch [35/100], Loss: 0

## ResNet18 + MLP

In [ ]:
frame_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def load_video_frames(video_path, max_frames=16):
    frames = sorted(video_path.glob("*.jpg"))[:max_frames]
    video_tensor = torch.stack([frame_transform(Image.open(f).convert("RGB")) for f in frames])
    if len(frames) < max_frames:
        padding = torch.zeros((max_frames - len(frames), 3, 224, 224))
        video_tensor = torch.cat([video_tensor, padding], dim=0)
    return video_tensor

X = []
y = []

for _, row in df.iterrows():
    video_id = row["VIDEO"]
    video_dir = FRAME_DIR / video_id
    if not video_dir.exists():
        continue
    video_tensor = load_video_frames(video_dir, max_frames=16)
    labels = torch.tensor([
        row["OSATS_RESPECT"], row["OSATS_MOTION"], row["OSATS_INSTRUMENT"],
        row["OSATS_SUTURE"], row["OSATS_FLOW"], row["OSATS_KNOWLEDGE"],
        row["OSATS_PERFORMANCE"], row["OSATS_FINAL_QUALITY"]
    ], dtype=torch.long)
    X.append(video_tensor)
    y.append(labels)

X = torch.stack(X)
y = torch.stack(y)

# Corrigir labels se estiverem na escala 1–5 (passar para 0–4)
if torch.any(y > 4):
    print("Corrigindo escala de labels de 1–5 para 0–4...")
    y = y - 1

# Verificação de segurança
assert torch.all((y >= 0) & (y <= 4)), "Erro: targets fora do intervalo 0–4"


Corrigindo escala de labels de 1–5 para 0–4...


In [ ]:
from torch.utils.data import random_split

class OSATSDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = OSATSDataset(X, y)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
class OSATSResNet(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = models.resnet18(pretrained=True)
        resnet.fc = nn.Identity()
        self.backbone = resnet
        self.fc_shared = nn.Linear(512, 128)
        self.heads = nn.ModuleList([nn.Linear(128, 5) for _ in range(8)])

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)
        feat = self.backbone(x).view(B, T, -1)
        feat = feat.mean(dim=1)
        shared = F.relu(self.fc_shared(feat))
        return torch.stack([head(shared) for head in self.heads], dim=1)


In [ ]:
loss_fn = nn.CrossEntropyLoss()

def compute_loss(preds, targets):
    loss = 0
    for i in range(8):
        loss += loss_fn(preds[:, i], targets[:, i])
    return loss / 8


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = OSATSResNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    all_preds, all_targets = [], []

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = compute_loss(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        preds = torch.argmax(outputs, dim=2)
        all_preds.append(preds.cpu())
        all_targets.append(targets.cpu())

    avg_loss = train_loss / len(train_loader.dataset)
    print(f"Época {epoch+1}/{num_epochs} - Loss treino: {avg_loss:.4f}")

# Guardar o modelo após treino
torch.save(model.state_dict(), "../outputs/models/model_task2_resnet.pt")


/home/luis/miniconda3/envs/ap/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/luis/miniconda3/envs/ap/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Época 1/100 - Loss treino: 1.6557
Época 2/100 - Loss treino: 1.3940
Época 3/100 - Loss treino: 1.2120
Época 4/100 - Loss treino: 1.0552
Época 5/100 - Loss treino: 0.9106
Época 6/100 - Loss treino: 0.7806
Época 7/100 - Loss treino: 0.6666
Época 8/100 - Loss treino: 0.5685
Época 9/100 - Loss treino: 0.4842
Época 10/100 - Loss treino: 0.4102
Época 11/100 - Loss treino: 0.3460
Época 12/100 - Loss treino: 0.2908
Época 13/100 - Loss treino: 0.2446
Época 14/100 - Loss treino: 0.2060
Época 15/100 - Loss treino: 0.1730
Época 16/100 - Loss treino: 0.1453
Época 17/100 - Loss treino: 0.1220
Época 18/100 - Loss treino: 0.1022
Época 19/100 - Loss treino: 0.0854
Época 20/100 - Loss treino: 0.0716
Época 21/100 - Loss treino: 0.0603
Época 22/100 - Loss treino: 0.0510
Época 23/100 - Loss treino: 0.0433
Época 24/100 - Loss treino: 0.0371
Época 25/100 - Loss treino: 0.0319
Época 26/100 - Loss treino: 0.0276
Época 27/100 - Loss treino: 0.0241
Época 28/100 - Loss treino: 0.0211
Época 29/100 - Loss treino: 0

## ResNet18 + GRU

In [ ]:
class OSATSResNetGRU(nn.Module):
    def __init__(self, hidden_size=128, num_layers=1):
        super().__init__()
        resnet = models.resnet18(pretrained=True)
        resnet.fc = nn.Identity()
        self.backbone = resnet  

        self.gru = nn.GRU(input_size=512, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.heads = nn.ModuleList([nn.Linear(hidden_size, 5) for _ in range(8)])  

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)
        features = self.backbone(x)  
        features = features.view(B, T, 512) 

        _, h_n = self.gru(features) 
        h_n = h_n.squeeze(0) 

        return torch.stack([head(h_n) for head in self.heads], dim=1)  


In [ ]:
def compute_loss(preds, targets):
    loss_fn = nn.CrossEntropyLoss()
    total_loss = 0.0

    for i in range(preds.size(1)):  # 8 cabeças
        total_loss += loss_fn(preds[:, i], targets[:, i])
    
    return total_loss / preds.size(1)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = OSATSResNetGRU().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    train_loss = 0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = compute_loss(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)

    avg_loss = train_loss / len(train_loader.dataset)
    print(f"Época {epoch+1}/{num_epochs} - Loss treino: {avg_loss:.4f}")

# Guardar o modelo
torch.save(model.state_dict(), "../outputs/models/model_task2_resnet_gru.pt")


Época 1/100 - Loss treino: 1.6299
Época 2/100 - Loss treino: 1.0909
Época 3/100 - Loss treino: 0.8264
Época 4/100 - Loss treino: 0.6919
Época 5/100 - Loss treino: 0.6110
Época 6/100 - Loss treino: 0.5569
Época 7/100 - Loss treino: 0.5152
Época 8/100 - Loss treino: 0.4811
Época 9/100 - Loss treino: 0.4531
Época 10/100 - Loss treino: 0.4297
Época 11/100 - Loss treino: 0.4091
Época 12/100 - Loss treino: 0.3904
Época 13/100 - Loss treino: 0.3731
Época 14/100 - Loss treino: 0.3571
Época 15/100 - Loss treino: 0.3432
Época 16/100 - Loss treino: 0.3299
Época 17/100 - Loss treino: 0.3176
Época 18/100 - Loss treino: 0.3064
Época 19/100 - Loss treino: 0.2957
Época 20/100 - Loss treino: 0.2857
Época 21/100 - Loss treino: 0.2763
Época 22/100 - Loss treino: 0.2673
Época 23/100 - Loss treino: 0.2587
Época 24/100 - Loss treino: 0.2504
Época 25/100 - Loss treino: 0.2426
Época 26/100 - Loss treino: 0.2351
Época 27/100 - Loss treino: 0.2280
Época 28/100 - Loss treino: 0.2212
Época 29/100 - Loss treino: 0

## Vision Transformer (ViT) + MLP

In [ ]:
class OSATSViT(nn.Module):
    def __init__(self):
        super().__init__()
        vit = models.vit_b_16(pretrained=True)
        vit.heads = nn.Identity()  # remover a cabeça original
        self.backbone = vit  # saída: [B*T, 768]

        self.shared = nn.Linear(768, 128)
        self.heads = nn.ModuleList([nn.Linear(128, 5) for _ in range(8)])

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)             # [B*T, 3, 224, 224]
        features = self.backbone(x)            # [B*T, 768]
        features = features.view(B, T, -1)     # [B, T, 768]
        pooled = features.mean(dim=1)          # média temporal → [B, 768]

        shared = F.relu(self.shared(pooled))   # [B, 128]
        return torch.stack([head(shared) for head in self.heads], dim=1)  # [B, 8, 5]


In [ ]:
def compute_loss(preds, targets):
    loss_fn = nn.CrossEntropyLoss()
    total_loss = 0.0

    for i in range(preds.size(1)):  # 8 cabeças
        total_loss += loss_fn(preds[:, i], targets[:, i])
    
    return total_loss / preds.size(1)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = OSATSViT().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    train_loss = 0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = compute_loss(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)

    avg_loss = train_loss / len(train_loader.dataset)
    print(f"Época {epoch+1}/{num_epochs} - Loss treino: {avg_loss:.4f}")

# Guardar o modelo
torch.save(model.state_dict(), "../outputs/models/model_task2_vit.pt")


/home/luis/miniconda3/envs/ap/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /home/luis/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [11:05<00:00, 520kB/s] 


## Evaluation

In [ ]:
def validate_model_multihead(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_items = 0

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)  # [B, 8, 5]
            loss = criterion(outputs.view(-1, 5), targets.view(-1))
            total_loss += loss.item() * inputs.size(0)

            preds = outputs.argmax(dim=2)  # [B, 8]
            total_correct += (preds == targets).sum().item()
            total_items += targets.numel()

    avg_loss = total_loss / len(dataloader.dataset)
    avg_acc = total_correct / total_items
    return avg_loss, avg_acc


In [ ]:
model1 = CNNModel_1().to(device)
model1.load_state_dict(torch.load("../outputs/models/model_task2_cnn1.pt"))

model2 = CNNModel_2().to(device)
model2.load_state_dict(torch.load("../outputs/models/model_task2_cnn2.pt"))

model3 = CNNModel_3().to(device)
model3.load_state_dict(torch.load("../outputs/models/model_task2_cnn3.pt"))

model4 = CNNModel_4().to(device)
model4.load_state_dict(torch.load("../outputs/models/model_task2_cnn4.pt"))

model5 = OSATSResNet().to(device)
model5.load_state_dict(torch.load("../outputs/models/model_task2_resnet.pt"))

model6 = OSATSResNetGRU().to(device)
model6.load_state_dict(torch.load("../outputs/models/model_task2_resnet_gru.pt"))

model7 = OSATSViT().to(device)
model7.load_state_dict(torch.load("../outputs/models/model_task2_vit.pt"))

models = {
    "CNNModel_1": model1,
    "CNNModel_2": model2,
    "CNNModel_3": model3,
    "CNNModel_4": model4,
    "ResNet + MLP": model5,
    "ResNet + GRU": model6,
    "ViT + MLP": model7
}


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

# Gráfico de Loss
plt.subplot(1, 2, 1)
plt.plot([h[0] for h in hist1], label='CNNModel_1 Loss')
plt.plot([h[0] for h in hist2], label='CNNModel_2 Loss')
plt.plot([h[0] for h in hist3], label='CNNModel_3 Loss')
plt.plot([h[0] for h in hist4], label='CNNModel_4 Loss')
plt.plot([h[0] for h in hist5], label='CNNLSTM Loss')
plt.plot([h[0] for h in hist6], label='EfficientNetLSTM Loss')
plt.plot([h[0] for h in hist7], label='ViTLSTM Loss')
plt.title('Loss durante treino')
plt.xlabel('Época')
plt.ylabel('Loss')
plt.legend()
plt.grid()

# Gráfico de Accuracy
plt.subplot(1, 2, 2)
plt.plot([h[1] for h in hist1], label='CNNModel_1 Accuracy')
plt.plot([h[1] for h in hist2], label='CNNModel_2 Accuracy')
plt.plot([h[1] for h in hist3], label='CNNModel_3 Accuracy')
plt.plot([h[1] for h in hist4], label='CNNModel_4 Accuracy')
plt.plot([h[1] for h in hist5], label='CNNLSTM Accuracy')
plt.plot([h[1] for h in hist6], label='EfficientNetLSTM Accuracy')
plt.plot([h[1] for h in hist7], label='ViTLSTM Accuracy')
plt.title('Accuracy durante treino')
plt.xlabel('Época')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()


In [ ]:
criterion = nn.CrossEntropyLoss()
val_results = {}

for name, model in models.items():
    val_loss, val_acc = validate_model_multihead(model, val_loader, criterion)
    val_results[name] = val_acc
    print(f"{name} - Val Acc: {val_acc:.4f}")


In [ ]:
best_model_name = max(val_results, key=val_results.get)
best_model = models[best_model_name]
print(f"Melhor modelo: {best_model_name} com accuracy {val_results[best_model_name]:.4f}")

test_loss, test_acc = validate_model_multihead(best_model, test_loader, criterion)
print(f"Acurácia no Teste: {test_acc:.4f}")


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

osats_cols = [col for col in df.columns if col.startswith("OSATS_")]

best_model.eval()
all_true, all_pred = [], []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = best_model(inputs)
        preds = outputs.argmax(dim=2)

        all_true.append(targets.cpu())
        all_pred.append(preds.cpu())

true = torch.cat(all_true)
pred = torch.cat(all_pred)

for i, col in enumerate(osats_cols):
    cm = confusion_matrix(true[:, i], pred[:, i])
    plt.figure(figsize=(4, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(f"Matriz de Confusão - {col}")
    plt.xlabel("Predito")
    plt.ylabel("Verdadeiro")
    plt.show()
